In [1]:
import oracledb, os, json
from google.cloud import secretmanager
import pandas as pd
import numpy as np
import paramiko
import datetime
from io import StringIO

import warnings
warnings.filterwarnings('ignore')

In [2]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [3]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    sftpkey = os.getenv('SFTPKEY'),
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def do_etl(periode):
    keyfile = StringIO(oracle_secrets['sftpkey'])
    mykey = paramiko.RSAKey.from_private_key(keyfile, password=np.nan)

    host,port = "a01drvl099.adeo.no",22
    transport = paramiko.Transport((host,port))

    username= "srv-dv-familie-airflow-sas"
    transport.connect(username=username,pkey=mykey)

    with paramiko.SFTPClient.from_transport(transport) as sftp:
        local_dir = 'data'
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)
        sftp.get(f'./inbound/kildefiler/bidrag/BIDRAG_STONAD_M{periode}', f'data/BIDRAG_STONAD_M{periode}')

    if sftp: sftp.close()
    if transport: transport.close()

In [21]:
def get_periode():
    today = datetime.date.today() # dato for idag 2022-04-19
    first = today.replace(day=1) # dato for første dag i måneden 2022-04-01
    lastMonth = first - datetime.timedelta(days=1) # dato for siste dag i tidligere måneden

    return lastMonth.strftime("%Y%m") # henter bare aar og maaned

In [22]:
periode = get_periode()

do_etl(periode)

In [ ]:
%%bash

per=$(python -c 'import datetime; print((datetime.date.today().replace(day=1)-datetime.timedelta(days=1)).strftime("%Y%m"))')

cat data/BIDRAG_STONAD_M$per | tr -d '\000' > data/BIDRAG_STONAD_M$per.txt

In [23]:
barn_df = pd.read_excel('posisjoner/barn_stonad_pos.xlsx',header=0 )
barn_df.rename(columns={"Variabel-navn": "feltnavn", "Start-pos. i Bidrag-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

mottaker_df = pd.read_excel('posisjoner/mottaker_stonad_pos.xlsx',header=0 )
mottaker_df.rename(columns={"Feltnavn": "feltnavn", "Start-pos. i Bidrag-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

pliktig_df = pd.read_excel('posisjoner/pliktig_stonad_pos.xlsx',header=0 )
pliktig_df.rename(columns={"Feltnavn": "feltnavn", "Start-pos. i Bidrag-fila": "start_pos", "Antall posi-sjoner": "antall_pos"}, inplace = True)

In [ ]:
def do_tl():

    user = oracle_secrets['user'] + '[DVH_FAM_BB]'
    dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])

    df_stonad = pd.read_csv(f"data/BIDRAG_STONAD_M{periode}.txt", skip_blank_lines=True, sep='\0', header=None)
    
    df_stonad['REC_TYPE'] = df_stonad[0].apply(lambda x: x[13:15])
    filt = (df_stonad['REC_TYPE'] == '03')
    df_rec_type_3 = df_stonad.loc[filt].copy()

    for index, row in barn_df.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        #print(row['feltnavn'],start_pos, end_pos)
        df_rec_type_3[row['feltnavn']] = df_rec_type_3[0].apply(lambda x: x[start_pos:end_pos])
        
    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
    df_barn = df_rec_type_3[barn_df['feltnavn']]
    df_barn = df_barn.replace(r'^\s*$', None, regex=True)

    filt = (((df_barn['FORSKUDD'].fillna(0).astype(int) > 0)|(df_barn['BIDRAG_G'].fillna(0).astype(int) > 0) ))
    df_barn = df_barn.loc[filt].copy()

    df_stonad['REC_TYPE'] = df_stonad[0].apply(lambda x: x[13:15])
    filt = (df_stonad['REC_TYPE'] == '02')
    df_rec_type_2 = df_stonad.loc[filt].copy()

    for index, row in mottaker_df.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        df_rec_type_2[row['feltnavn']] = df_rec_type_2[0].apply(lambda x: x[start_pos:end_pos])

    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
    df_mottaker = df_rec_type_2[mottaker_df['feltnavn']]
    df_mottaker = df_mottaker.replace(r'^\s*$', None, regex=True)

    # Filter df_mottaker based on SAKSNR in df_barn
    df_mottaker = df_mottaker[df_mottaker['SAKSNR'].isin(df_barn['SAKSNR'])]

    df_stonad['REC_TYPE'] = df_stonad[0].apply(lambda x: x[13:15])
    filt = (df_stonad['REC_TYPE'] == '01')
    df_rec_type_1 = df_stonad.loc[filt].copy()

    for index, row in pliktig_df.iterrows():
        start_pos = row['start_pos'] - 1
        end_pos = start_pos + row['antall_pos']
        df_rec_type_1[row['feltnavn']] = df_rec_type_1[0].apply(lambda x: x[start_pos:end_pos])

    # henter kun kolonnene vi er intersert i ved bruk av kolonne "Feltnavn" fra mottaker_df som inneholder navn på alle kolonner
    df_pliktig = df_rec_type_1[pliktig_df['feltnavn']]
    df_pliktig = df_pliktig.replace(r'^\s*$', None, regex=True)

    # Filter df_mottaker based on SAKSNR in df_barn
    df_pliktig = df_pliktig[df_pliktig['SAKSNR'].isin(df_barn['SAKSNR'])]

    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            rows = [tuple(x) for x in df_barn.values]
            cursor.executemany('''INSERT INTO STG_FAM_BB_BARN_BIS(PERIODE,SAKSNR,REC_TYPE,OBJNR,TKNR_BOST,FNR,ALDER,KJONN,INSTITUSJON,BIDRAG_G,FORSK,SBBEL,SUM_P_KRAV,F_FORSKUDD,ETTERG_BEL
                                ,ANT_ETTERG,MOID,MOTYPE,SAKSTYP,SAKSDATO,BOLAND,BPFNR,BMFNR,FORSKUDD,HGBOST,UGBOST,RESKODE,SBBELHI,BIDRAG
                                ,LOPSAK,UTLVAL,VEDTDATOBB,VIRKDATOBB,VEDTDATOFO,VIRKDATOFO) 
                                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,
                                    :25,:26,:27,:28,:29,:30,:31,:32,:33,:34,:35)''',rows)
        with conn.cursor() as cursor:
            rows = [tuple(x) for x in df_mottaker.values]
            cursor.executemany('''INSERT INTO STG_FAM_BB_MOTTAKER_BIS(PERIODE,SAKSNR,REC_TYPE,TKNR_BOST,FNR,ALDER,KJONN,BOLAND,STONAD_STAT
                                ,SAKSTYP,EKBIDR_G,OBJNR,ANTBARNB,EKBIDR ,BIDRSUM,LOPSAK,UTLVAL ,LAND) 
                                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18)''',rows)
        with conn.cursor() as cursor:   
            rows = [tuple(x) for x in df_pliktig.values]
            cursor.executemany('''INSERT INTO STG_FAM_BB_PLIKTIGE_BIS(PERIODE,SAKSNR,REC_TYPE,TKNR_BOST,FNR,ALDER,KJONN,BOLAND,INNB_TOTALT,INNB_UFORDELT,INNB_11_21,INNB_32
                                ,INNB_42,INNB_31_41,INNB_12_22,F_PAALOP,SUM_PAALOP,SUM_GJELD,GJELD_11_21,GJELD_32
                                ,GJELD_42,GJELD_31_41,GJELD_12_22,LOPSAK,SAKSTYP,OBJNR,ANTBARNB,BIDRSUM,UTLVAL,LAND) 
                                    VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17,:18,:19,:20,:21,:22,:23,:24,
                                    :25,:26,:27,:28,:29,:30)''',rows)
            conn.commit()


In [28]:
do_tl()